In [5]:
#import necessary tools
import re
from string import digits
import pandas as pd
import numpy as np

In [11]:
#read the documents
with open('issue/frate1_211BL.txt','rb') as f:
    contents = f.read()
    contents = contents.decode("utf-16").rstrip("\n")
    contents = contents.split("\r\n") #maybe need to strip \t?

In [12]:
Procedure=list()
img =[]
Choice_RESP = []
Choice_RT=[]


#loop through each line
for line in range(len(contents)-20):
    
    #this line of "running: listlike/black" is in the fixed position of each of 4 conditions (b1b2l1l2)
    #therefore, we use it as our anchor to locate meaningful blocks

    like_check = contents[line+5].startswith("\tRunning: ListRate") == True
    #black_check = contents[line+5].startswith("\tRunning: ListDark") == True
    
    if contents[line] == '\t*** LogFrame Start ***' and like_check:
        
        #loop inside the meaningful block                
        for inner_line in range(20):
            
            new_line = line + inner_line
            
            #mark image
            if contents[new_line].startswith("\timg: "):
                imga = contents[new_line].split("img: ",1)[1] 
                #print("imag is", imga)
                img.append(imga)
                
             
            #mark choice Like response 
            if contents[new_line].startswith("\tSlideR.RESP: ") and contents[new_line].split("\tSlideR.RESP: ",1) != ['', '']:

                Choice_RESPa = contents[new_line].split("\tSlideR.RESP: ",1)[1]
                Choice_RESPa =[''.join(c for c in Choice_RESPa if c in digits)]
                Choice_RESP=Choice_RESP+Choice_RESPa
                
            #mark choice Like response time
            if contents[new_line].startswith("\tSlideR.RT: ") and contents[new_line].split("\tSlideR.RT: ",1) != ['']:
                
                Choice_RTa = contents[new_line].split("\tSlideR.RT: ",1)[1]
                Choice_RTa =[''.join(c for c in Choice_RTa if c in digits)]
                Choice_RTa = list([float(i) for i in Choice_RTa])
                Choice_RT=Choice_RT+Choice_RTa


#Assign variable names  
        
img = [int(i) for i in img] 
img_df = pd.DataFrame(img)
img_df.columns = ['imgL1']

Choice_RESP = [int(i) for i in Choice_RESP] 
Choice_RESP_df = pd.DataFrame(Choice_RESP)
Choice_RESP_df.columns = ['rateL1']

Choice_RT = [int(i) for i in Choice_RT] 
Choice_RT_df = pd.DataFrame(Choice_RT)
Choice_RT_df.columns = ['rtL1']


ValueError: Length mismatch: Expected axis has 0 elements, new values have 1 elements

In [13]:
#merge the data
sub_id = contents[16].split("Subject: ",1)[1] 
sub_df = pd.DataFrame(np.repeat(sub_id, img_df.shape[0], axis = 0))
sub_df.columns = ['sub']


all = pd.concat([sub_df.reset_index(drop=True), img_df], axis=1)
all = pd.concat([all.reset_index(drop=True), Choice_RESP_df], axis=1)
all = pd.concat([all.reset_index(drop=True), Choice_RT_df], axis=1)

all

,sub,rateL1,rtL1
0,NaN,3,3782
1,NaN,1,1788
2,NaN,2,1714
3,NaN,4,3264
4,NaN,2,2381
...,...,...,...
115,NaN,8,2605
116,NaN,1,2460
117,NaN,3,2371
118,NaN,8,2578


In [9]:
#remove last NA line if needed

#all_all= all[:-1]

all_all= all


all_all

,sub,imgL1,rateL1,rtL1
0,108,121,3,3782
1,108,122,1,1788
2,108,123,2,1714
3,108,124,4,3264
4,108,125,2,2381
...,...,...,...,...
115,108,236,8,2605
116,108,237,1,2460
117,108,238,3,2371
118,108,239,8,2578


In [10]:

#export
all_all.to_excel("frate1_108.xlsx")

#has wrong first column, needs to manually remove